In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"

Running local


In [2]:
import os
import numpy as np

import datetime
from datetime import timedelta

import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from Zamuro_DataLoader import SoundscapeData

import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

def find_wav_filenames( path_to_dir, suffix=".wav" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

In [ ]:
files_Ais = find_csv_filenames(f"{root}/Zamuro/AI_Caract")
file_rain = f"{root}/Zamuro/Complementary_Files/Result_rain_merged.xlsx"
files_udas = f"{root}/Zamuro/Complementary_Files/UDAS Pasivo_20221001_Zamuro.xlsx"
files_covers = f"{root}/Zamuro/Complementary_Files/ReyZamuro_softLabels.csv"
files_recorders = os.listdir("/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/")
path_recorders = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/"

In [ ]:
UDAS = pd.read_excel(f"{files_udas}", sheet_name="Template")
covers = pd.read_csv(f"{files_covers}")
# AIs = pd.read_csv(files_Ais)
habitat = ["bosque"]* 2 + ["pastizal"] + ["bosque"] + ["borde"] * 2 + ["bosque"] +  ["parche"] *2 + ["bosque"]*4 + ["borde"]*11 + ["bosque"] * 9 + ["parche"]*2 + ["pastizal"]*4 + ["parche"]*2 + ["sabana"]*10 + ["borde"] + ["sabana"] + ["pastizal"] + ["sabana"] + ["bosque"]*3 + ["pastizal"]*2 + ["potrero"] * 2 + ["bosque"] + ["borde"]* 5 +["bosque"] * 9 + ["sabana"]* 2 + ["pastizal"] + ["parche"]
habitat = [
    "bosque", "bosque", "pastizal", "borde", "borde", "borde", "bosque", "sabana", "bosque", "borde",
    "bosque", "bosque", "bosque", "bosque", "bosque", "borde", "borde", "pastizal", "potrero", "sabana",
    "borde", "bosque", "bosque", "sabana", "borde", "bosque", "borde", "bosque", "bosque", "bosque", "borde",
    "potrero", "bosque", "bosque", "sabana", "bosque", "bosque", "sabana", "parche", "bosque", "pastizal",
    "bosque", "bosque", "borde", "bosque", "borde", "borde", "bosque", "parche", "bosque", "bosque", "sabana",
    "bosque", "pastizal", "pastizal", "sabana", "sabana", "borde", "bosque", "sabana", "bosque", "sabana",
    "bosque", "sabana", "borde", "borde", "borde", "borde", "pastizal", "bosque", "pastizal", "pastizal",
    "bosque", "bosque", "bosque", "bosque", "sabana", "pastizal", "parche", "borde", "sabana", "sabana", "parche",
    "parche", "sabana", "borde", "sabana", "parche", "bosque", "sabana", "parche", "bosque", "sabana", "borde",
    "borde", "sabana", "sabana", "borde", "bosque", "bosque", "bosque", "bosque", "bosque", "bosque", "bosque", "bosque"
]

keep_cols = ["field_number_PR", "longitud_IG", "latitude_IG", "day_init_TI", "day_end_TI", "samp_rate_RE"]

In [ ]:
UDAS = UDAS[keep_cols]
UDAS["habitat"] = habitat
UDAS.info()
covers.rename(columns={"field_numb":"field_number_PR", "1":"pasture", "2": "savanna", "3": "forest"}, inplace=True)
a=UDAS
b=covers
merged_df = a.merge(b, on='field_number_PR', how='left')
result_df = merged_df[merged_df['pasture'].notna()]
result_df.reset_index(inplace=True)
result_df.drop("index", axis=1, inplace=True)
result_df["cover"] = result_df[["pasture", "savanna", "forest"]].idxmax(axis=1)
result_df.to_csv(f"{root}//Zamuro/Complementary_Files/zamuro_recorders.csv")

In [ ]:
audios = pd.DataFrame(columns=result_df.columns)
total_files=0
for i in range(len(result_df)):
    aux = pd.DataFrame(columns=result_df.columns)
    path = root_path+"/"+result_df.loc[i, "field_number_PR"]
    files = os.listdir(path)
    total_files+=len(files)
    aux["Filename"] = files
    name=[i.split('.', 1)[0] for i in files]
    aux["name_FI"] = name
    for col in aux.columns[0:-2]:
        aux[col] = result_df.loc[i,col]
    audios = pd.concat([audios, aux])
audios.to_csv(f"{root}/Zamuro/Complementary_Files/zamuro_audios.csv")
audios.reset_index(inplace=True)

In [ ]:
rain = pd.read_excel(file_rain)
rain = rain[["field_number_PR", "name_FI","rain_FI"]]
audios["rain_FI"] = "-"
count = 0
for i in range(len(audios)):
    if i%10000 == 0:
        print(i)
    try:
        audio = audios.loc[i]
        audios.loc[i, "rain_FI"] = rain.loc[rain["field_number_PR"] == audio["field_number_PR"]].loc[rain["name_FI"] == audio["Filename"]]["rain_FI"].values[0]
    except:
        count+=1
audios.to_csv(f"{root}//Zamuro/Complementary_Files/zamuro_audios.csv")

In [4]:
audios_zamuro = pd.read_csv(f"{root}//Zamuro/Complementary_Files/zamuro_audios.csv")
audios_zamuro.drop(columns=["Unnamed: 0"], inplace=True)
set(audios_zamuro["field_number_PR"])

{'RZUA01',
 'RZUA02',
 'RZUA03b',
 'RZUA04',
 'RZUA05',
 'RZUA06',
 'RZUA07',
 'RZUA08',
 'RZUA09',
 'RZUA10',
 'RZUB01',
 'RZUB02',
 'RZUB03',
 'RZUB04',
 'RZUB05',
 'RZUB06',
 'RZUB07',
 'RZUB08',
 'RZUB09',
 'RZUB10',
 'RZUB11',
 'RZUC01',
 'RZUC02',
 'RZUC03',
 'RZUC05',
 'RZUC06',
 'RZUC07',
 'RZUC08',
 'RZUC09',
 'RZUC10',
 'RZUC11',
 'RZUC12',
 'RZUD01M',
 'RZUD02',
 'RZUD03',
 'RZUD04',
 'RZUD05',
 'RZUD06',
 'RZUD07',
 'RZUD08',
 'RZUD09',
 'RZUD10',
 'RZUD11',
 'RZUD12',
 'RZUD13',
 'RZUE01',
 'RZUE02',
 'RZUE03',
 'RZUE04',
 'RZUE05',
 'RZUE06',
 'RZUE07',
 'RZUE08',
 'RZUE09',
 'RZUE10',
 'RZUE11',
 'RZUE12',
 'RZUE13',
 'RZUF02',
 'RZUF03',
 'RZUF04',
 'RZUF05',
 'RZUF06',
 'RZUF07',
 'RZUF08',
 'RZUF09',
 'RZUF10',
 'RZUF11',
 'RZUF12',
 'RZUF13',
 'RZUG02',
 'RZUG03',
 'RZUG04',
 'RZUG05',
 'RZUG06',
 'RZUG07',
 'RZUG08',
 'RZUG09',
 'RZUG10',
 'RZUG11',
 'RZUG12',
 'RZUG13',
 'RZUH02',
 'RZUH03',
 'RZUH04',
 'RZUH05',
 'RZUH06',
 'RZUH07',
 'RZUH08',
 'RZUH09',
 'RZUH10

In [ ]:
lista = list(set(audios_zamuro["field_number_PR"]))
lista.sort()
for csv in lista:
    temporal_csv = audios_zamuro.loc[audios_zamuro["field_number_PR"] == csv]
    temporal_csv.drop(columns=["index"],inplace=True)
    temporal_csv.to_csv(f"{root}//Zamuro/Complementary_Files/{csv}.csv",
                        index=True, index_label="global_index")

In [48]:
model = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 4
hour = 9
date_format = f"day_{day}_hour_{hour}"
X = torch.load(f"temporal_zamuro/Features/AE_features_{date_format}_rain_NO_{lista[54]}.csv.pth",  map_location=torch.device('cpu'))

In [192]:
X = torch.load(f"temporal_zamuro/Features/AE_features_{date_format}_rain_NO_{lista[0]}.csv.pth",  map_location=torch.device('cpu'))
y = torch.load(f"temporal_zamuro/Features/AE_test_path_samples_{date_format}_rain_NO_{lista[0]}.csv.pth",  map_location=torch.device('cpu'))
for i, csv in enumerate(lista[1::]):
    X_i = torch.load(f"temporal_zamuro/Features/AE_features_{date_format}_rain_NO_{csv}.csv.pth",  map_location=torch.device('cpu'))
    y_i = torch.load(f"temporal_zamuro/Features/AE_test_path_samples_{date_format}_rain_NO_{csv}.csv.pth",  map_location=torch.device('cpu'))
    X = torch.cat((X,X_i),0)
    y = y + y_i
torch.save(X, f"temporal_zamuro/Features/AE_features_Zamuro.pth")
torch.save(y, f"temporal_zamuro/Features/AE_test_path_samples_Zamuro.pth")
       
path_error = torch.load(f"temporal_zamuro/Features/AE_test_corrupted_samples_list_{date_format}_rain_NO_{lista[9]}.csv.pth",  map_location=torch.device('cpu'))
y_path = torch.load(f"temporal_zamuro/Features/AE_test_path_samples_{date_format}_rain_NO_{lista[9]}.csv.pth",  map_location=torch.device('cpu'))

In [215]:
path_flat = [item for sublist in y_path for item in sublist]
path_flat = np.asarray(path_flat)

In [219]:
len(y_path*5)

4270

In [217]:
path_flat.shape

(4270,)

In [195]:
y_0 = torch.load(f"temporal_zamuro/Features/AE_test_path_samples_{date_format}_rain_NO_{lista[0]}.csv.pth",  map_location=torch.device('cpu'))
y_1 = torch.load(f"temporal_zamuro/Features/AE_test_path_samples_{date_format}_rain_NO_{lista[1]}.csv.pth",  map_location=torch.device('cpu'))
y = torch.load(f"temporal_zamuro/Features/AE_test_path_samples_Zamuro.pth",  map_location=torch.device('cpu'))

In [94]:
audios_zamuro_clean = audios_zamuro.copy()